# <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>Python 基础系列</font></div>
## <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>装饰器</font></div>

## <font color='#dc2624' face='微软雅黑'>目录</font><a name='toc'></a>
### 1. [**<font color='#dc2624' face='微软雅黑'>装饰器 101</font>**](#1)
1. [<font color='#2b4750' face='微软雅黑'>简单例子</font>](#1.1)
2. [<font color='#2b4750' face='微软雅黑'>闭包</font>](#1.2)
3. [<font color='#2b4750' face='微软雅黑'>第一个装饰器</font>](#1.3)

### 2. [**<font color='#dc2624' face='微软雅黑'>装饰器知识点</font>**](#2)
1. [<font color='#2b4750' face='微软雅黑'>多个装饰器</font>](#2.1)
2. [<font color='#2b4750' face='微软雅黑'>装饰函数传入参数</font>](#2.2)
3. [<font color='#2b4750' face='微软雅黑'>重新找回身份</font>](#2.3)
4. [<font color='#2b4750' face='微软雅黑'>装饰器传入参数</font>](#2.4)

### 3. [**<font color='#dc2624' face='微软雅黑'>装饰器实际案例</font>**](#3)
1. [<font color='#2b4750' face='微软雅黑'>查错</font>](#3.1)
2. [<font color='#2b4750' face='微软雅黑'>计时</font>](#3.2)
3. [<font color='#2b4750' face='微软雅黑'>日志</font>](#3.3)
---

# <font color='#dc2624' face='微软雅黑'>1. 装饰器 101</font><a name='1'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

### <font color='#2b4750' face='微软雅黑'>1.1 简单例子</font><a name='1.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

用鸡肉饼做汉堡。

In [1]:
def meat(food='--鸡肉饼--'):
    print(food)

In [2]:
burger = meat
burger()

--鸡肉饼--


哪有只是肉的汉堡？再加个加点蔬菜，用西红柿和沙拉菜来包着鸡肉。

In [3]:
def vegetable(func):
    def wrapper():
        print(' #西红柿# ')
        func()
        print(' ~沙拉菜~ ')
    return wrapper

In [4]:
burger = vegetable(meat)
burger()

 #西红柿# 
--鸡肉饼--
 ~沙拉菜~ 


再加两层面包就做成了汉堡了。

In [5]:
def bread(func):
    def wrapper():
        print('</------\>')
        func()
        print('<\------/>')
    return wrapper

In [6]:
burger = bread(vegetable(meat))
burger()

</------\>
 #西红柿# 
--鸡肉饼--
 ~沙拉菜~ 
<\------/>


由上面例子可知，面包和蔬菜「装饰」着鸡肉饼，`bread()` 和 `vegatable()` 这两个函数起着「装饰器」的作用，它们没有改变 `meat()` 函数，只在它的基础上添砖加瓦，最后把鸡肉饼装饰成汉堡。

如果用装饰器的正规语法，用 `@func` 语法，将 `@bread` 和 `@vegatable` 放在被装饰的函数上面。

In [11]:
@bread
@vegetable
def meat(food='--鸡肉饼--'):
    print(food)

In [12]:
burger = meat
burger()

</------\>
 #西红柿# 
--鸡肉饼--
 ~沙拉菜~ 
<\------/>


装饰器是有序的，如果互换 `bread()` 和 `vegatable()` 这两函数的位置，那么这汉堡最外层是蔬菜，中间是面包，里面是鸡肉饼，就不像汉堡了。

In [13]:
@vegetable
@bread
def meat(food='--鸡肉饼--'):
    print(food)

In [14]:
burger = meat
burger()

 #西红柿# 
</------\>
--鸡肉饼--
<\------/>
 ~沙拉菜~ 


### <font color='#2b4750' face='微软雅黑'>1.2 闭包</font><a name='1.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

在正式了解装饰器之前，我们回顾之前函数的知识点：

- **一等公民函数**：函数是可以赋值给变量，可以当成**参数**传给另一个函数，也可以在另一个函数中**返回**
- **闭包**：在嵌套函数 (外函数嵌套内函数) 中，内函数引用外函数的**非本地变量**，然后外函数返回内函数。

首先看一个闭包。内函数 `inner()` 可以使用外函数 `outer()` 的参数 `msg`, 注意 `msg` 不在内函数的定义范围内，而来自外函数的参数。最后外函数返回**内函数对象** `inner`。

In [23]:
def outer(msg):
    def inner():
        print(msg)
    return inner

在 `outer()` 中传递不同的参数 'Hi' 和 'Bye' 来定义不同的函数：

- `hi_func()` 输出 'Hi'
- `bye_func()` 输出 'Bye'

In [24]:
hi_func = outer('Hi')
print(hi_func)
hi_func()

<function outer.<locals>.inner at 0x000001D529B282F0>
Hi


In [25]:
bye_func = outer('Bye')
print(bye_func)
bye_func()

<function outer.<locals>.inner at 0x000001D529B289D8>
Bye


接下来，在上面函数中，

- 将 `outer()` 用 `decorator()` 来替换
- 将 `inner()` 用 `wrapper()` 来替换

In [26]:
def decorator(msg):
    def wrapper():
        print(msg)
    return wrapper

函数 `decorator()` 就**几乎**可当成装饰器了，返回 `wrapper` **函数对象** 等着被调用，一旦被调用就运行 `print(msg)` 而打印出 `msg`。

为什么说**几乎**呢？因为真正装饰器的参数是函数，而不是不同变量。将上面 `msg` 改成 `func`, `print(msg)` 改成 `return func()` 就是装饰器了。

In [27]:
def decorator(func):
    def wrapper():
        return func()
    return wrapper

### 因此，装饰器就是一个高阶函数，其输入是函数，其输出也是函数。

### <font color='#2b4750' face='微软雅黑'>1.3 第一个装饰器</font><a name='1.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

In [57]:
def greet():
    print('Hello!')

In [58]:
def decorator(func):
    def wrapper():
        print(f"Executed before function '{func.__name__}'")
        func()
        print(f"Executed after function '{func.__name__}'")
    return wrapper

In [59]:
greet()

Hello!


In [60]:
decorate_greet = decorator(greet)
decorate_greet()

Executed before function 'greet'
Hello!
Executed after function 'greet'


这样调用装饰器太过繁琐。将 `@decorator` 语法写在被装饰的函数上面，可以等价实现装饰函数的用法。

In [61]:
@decorator
def greet():
    print('Hello!')

In [62]:
greet()

Executed before function 'greet'
Hello!
Executed after function 'greet'


上面装饰器的等价用法

    greet = decorator(greet)
    
语法 `@decorator` 也称为是上面语句的语法糖。

# <font color='#dc2624' face='微软雅黑'>2. 装饰器知识点</font><a name='2'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

### <font color='#2b4750' face='微软雅黑'>2.1 多个装饰器</font><a name='2.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

可从多个方面装饰一个函数，而这需要多个装饰器来完成。

In [63]:
def slogan():
    return 'I love Python'

我们希望这句话

- 大写化 (定义个 `uppercase_decorator` 装饰器)
- 被分词 (定一个 `split_decorator` 装饰器)

代码如下：

In [64]:
def uppercase_decorator(func):
    def wrapper():
        return func().upper()
    return wrapper

In [65]:
def split_decorator(func):
    def wrapper():
        return func().split()
    return wrapper

按先「大写」再「分词」的顺序装饰 `slogan()`，注意装饰器是按**从下往上**的顺序。 

In [66]:
@split_decorator
@uppercase_decorator
def slogan():
    return 'I love Python'

In [67]:
# slogan = split_decorator(uppercase_decorator(slogan))
slogan()

['I', 'LOVE', 'PYTHON']

但如果按先「分词」再「大写」的顺序装饰 `slogan()`，就报错了，因为先用 `split()` 将句子分成列表，而列表中没有 `upper()` 方法。

In [68]:
@uppercase_decorator
@split_decorator
def slogan():
    return 'I love Python'

In [69]:
# slogan = uppercase_decorator(split_decorator(slogan))
slogan()

AttributeError: 'list' object has no attribute 'upper'

### <font color='#2b4750' face='微软雅黑'>2.2 装饰函数接受参数</font><a name='2.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

回顾上面装饰器的例子，首先明晰几个概念：

- `decorator(func)` 称为**装饰器**
- `wrapper()` 称为**装饰函数**
- `func` 称为**被装饰的函数**

在下面代码中，装饰函数 `wrapper()` 没有接受任何参数。

In [71]:
def decorator(func):
    def wrapper():
        print(f"Executed before function '{func.__name__}'")
        func()
        print(f"Executed after function '{func.__name__}'")
    return wrapper

那么该装饰器中被装饰的函数 `func()` 也一定没有参数，就像下面的 `greet()` 一样。

In [72]:
@decorator
def greet():
    print('Hello!')

In [73]:
greet()

Executed before function 'greet'
Hello!
Executed after function 'greet'


如果被装饰的函数有参数，那么用装饰器 `decorator` 来装饰它会报错。

In [74]:
@decorator
def add(a,b):
    return a+b

In [75]:
add(1,2)

TypeError: wrapper() takes 0 positional arguments but 2 were given

在装饰函数 `wrapper()` 里面定义两个参数，对应着被装饰的函数 `add(a,b)` 里面的 `a` 和 `b`，在里面调用原函数 `func(arg1,arg2)` 就没问题了。

In [83]:
def decorator(func):
    def wrapper( arg1, arg2 ):
        print(f"Executed before function '{func.__name__}'")
        print( func( arg1, arg2 ) )
        print(f"Executed after function '{func.__name__}'")
    return wrapper

In [84]:
@decorator
def add(a,b):
    return a+b

add(1,2)

Executed before function 'add'
3
Executed after function 'add'


但是当 `add()` 有 3 个参数就又会报错了。

In [85]:
@decorator
def add(a,b,c):
    return a+b+c

add(1,2,3)

TypeError: wrapper() takes 2 positional arguments but 3 were given

我们不能把参数写这么死。还记得 `*args` 和 `**kwargs` 可以接受无限个**位置参数**和**关键词参数**吗？

In [86]:
def decorator(func):
    def wrapper( *args, **kwargs ):
        print(f"Executed before function '{func.__name__}'")
        print( func( *args, **kwargs ) )
        print(f"Executed after function '{func.__name__}'")
    return wrapper

这样我们可以用装饰器 `decorator` 装饰任何函数了，拿 `add()` 小试牛刀一下。

In [87]:
@decorator
def add(a,b,c):
    return a+b+c

add(1,2,3)

Executed before function 'add'
6
Executed after function 'add'


In [88]:
@decorator
def add(a,b,c,d,f):
    return a+b+c+d+f

add(1,2,3,4,5)

Executed before function 'add'
15
Executed after function 'add'


### <font color='#2b4750' face='微软雅黑'>2.3 重新找回身份</font><a name='2.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

Python 里所有对象和函数在运行时可知自身 (introspect) 的属性，拿内置函数 `print` 举例，用 `__name__` 可以获取函数名称。

In [89]:
print

<function print>

In [90]:
print.__name__

'print'

上面描述对自定义的函数也成立，举例如下。

In [94]:
def greet():
    print('Hello!')

print(greet.__name__)

greet


然而当 `greet()` 函数被装饰之后，它的“身份”就丢了，看下面代码。

In [95]:
def decorator(func):
    def wrapper( *args, **kwargs ):
        print(f"Executed before function '{func.__name__}'")
        print( func( *args, **kwargs ) )
        print(f"Executed after function '{func.__name__}'")
    return wrapper

@decorator
def greet():
    print('Hello!')

In [96]:
print(greet.__name__)

wrapper


用 `@decorator` 装饰后的 greet()，再看它的名称已经变成了 `wrapper`。原因很简单，因为我们调用的是 `decorator(greet)`，而这个函数返回的确是 `wrapper()` 函数，因此名称是 `wrapper`。

上面的逻辑虽然没错，但打印出的 `wrapper` 信息通常没有什么用，我们还是希望原来的最开始的函数名称 - `greet`。

怎么操作？用 `functools` 包里的 `wraps`。

In [98]:
import functools

In [99]:
def decorator(func):
    @functools.wraps(func)
    def wrapper( *args, **kwargs ):
        print(f"Executed before function '{func.__name__}'")
        print( func( *args, **kwargs ) )
        print(f"Executed after function '{func.__name__}'")
    return wrapper

@decorator
def greet():
    print('Hello!')

In [100]:
print(greet.__name__)

greet


现在 `greet()` 函数的名称又变成了 `greet`。

### 最佳做法：在自己编写的所有装饰器中都使用 `functools.wraps`

### <font color='#2b4750' face='微软雅黑'>2.4 装饰器接受参数</font><a name='2.4'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

除了可以传递参数给装饰函数 `wrapper`，也可以传递参数给装饰器 `decorator`。

先看下面“只有装饰函数有参数”的例子，该装饰器 `money_format` 将数值保留到小数点后两位。 

In [4]:
def money_format(func):
    def wrapper(*args, **kwargs):
        return f'{func(*args, **kwargs):.2f}'
    return wrapper

In [5]:
@money_format
def add( PV1, PV2 ):
    return PV1 + PV2

In [6]:
add(100, 100.4545)

'200.45'

但这个 200.45 是以什么货币为单位呢？美元还是人民币? 这时我们可创建另一个装饰器，称为 `currency_unit`，并传递参数 `curr` 来区分货币单位。

In [7]:
def currency_unit(curr):
    def money_format(func):
        def wrapper(*args, **kwargs):
            return f'{func(*args, **kwargs):.2f}' + ' ' + curr
        return wrapper
    return money_format

In [8]:
@currency_unit('USD')
def add( PV1, PV2 ):
    return PV1 + PV2

add(100, 100.4545)

'200.45 USD'

In [9]:
@currency_unit('CNY')
def add( PV1, PV2, PV3 ):
    return PV1 + PV2 + PV3

add(100, 100, 100.4545)

'300.45 CNY'

# <font color='#dc2624' face='微软雅黑'>3. 装饰器实际案例</font><a name='3'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

不管多复杂的实际案例，使用装饰器都符合下面的基本格式。
```
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper
```

### <font color='#2b4750' face='微软雅黑'>3.1 查错</font><a name='3.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

In [112]:
import functools

def my_debugger(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        args_repr = [ repr(a) for a in args ]                      # 1
        kwargs_repr = [ f"{k}={v!r}" for k, v in kwargs.items() ]  # 2
        signature = ", ".join(args_repr + kwargs_repr)             # 3
        print( f"Calling {func.__name__}({signature})" )
        value = func(*args, **kwargs)
        print( f"{func.__name__!r} returned {value!r}" )           # 4
        return value
    return wrapper

这个装饰器用来查错 (debugging)，将被装饰函数 `func` 中的所有位置参数和关键字参数的。上面代码标注 #1 #2 #3 #4 的部分解释如下：

1. 将所有**位置参数**放在列表中，用 `repr()` 函数可将参数转换成一个漂亮字符串的形式。 
2. 将所有**关键字参数**放在列表中，在 f-string 中用 `{k}={v!r}` 得到每组`形参=实参`，`!r` 将实参转换成一个漂亮字符串的形式。
3. 用 `", ".join()` 将所有**位置参数**和**关键字参数**串联起来，中间用逗号分隔。
4. 打印出函数名以及返回结果。

In [134]:
@my_debugger
def instrument( id, ntl=1, curR='CNY', *args, **kwargs ):
    PV = 0
    for n in args:
        PV = PV + n
    return str(PV*ntl) + ' ' + curR

In [135]:
DCF = (1, 2, 3)
info = {'dc':'act/365', 'ctp':'GS'}
instrument( 'MM1001', 100, 'EUR', *DCF, **info )

Calling instrument('MM1001', 100, 'EUR', 1, 2, 3, dc='act/365', ctp='GS')
'instrument' returned '600 EUR'


'600 EUR'

如果要把装饰器用在内置函数上，用以下格式，拿 `math.factorial` 举例：

    math.factorial = my_debugger(math.factorial)

In [119]:
import math
math.factorial = my_debugger(math.factorial)

用泰勒公式来逼近自然指数 e

\begin{equation}
e = \sum_{i=0}^{\infty}\frac{1}{i!} = \frac{1}{0!} + \frac{1}{1!} + \frac{1}{2!} + \frac{1}{3!} + \cdots   
\end{equation}

In [120]:
def approximate_e(n=10):
    return sum(1 / math.factorial(i) for i in range(n))

In [121]:
approximate_e()

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24
Calling factorial(5)
'factorial' returned 120
Calling factorial(6)
'factorial' returned 720
Calling factorial(7)
'factorial' returned 5040
Calling factorial(8)
'factorial' returned 40320
Calling factorial(9)
'factorial' returned 362880


2.7182815255731922

对比 e 精确值：

In [123]:
math.exp(1)

2.718281828459045

### <font color='#2b4750' face='微软雅黑'>3.2 计时</font><a name='3.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

用该装饰器可以计量每个函数运行的时间。

In [124]:
import functools
import time

def my_timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        value = func(*args, **kwargs)
        run_time = time.time() - start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper

In [126]:
@my_timer
def tester(num_times=1):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [127]:
tester()

Finished 'tester' in 0.0129 secs


In [129]:
tester(1000)

Finished 'tester' in 7.7254 secs


### <font color='#2b4750' face='微软雅黑'>3.3 日志</font><a name='3.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

In [157]:
import functools
import logging
from datetime import datetime

def my_logger(func):
    
    logging.basicConfig( filename=f'{func.__name__}.log', level=logging.INFO )
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        logging.info( f'At {datetime.now()} - Ran with args: {args}, and kwargs: {kwargs}' )
        return func(*args, **kwargs)

    return wrapper

In [158]:
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print(f"'display_info' ran with arguments ({name}, {age})")

In [159]:
display_info( 'Tom', 22 )

'display_info' ran with arguments (Tom, 22)
Finished 'display_info' in 1.0003 secs


点开生成的日志文件 display_info.log，发现它已经记录了 Tom 的个人信息。

In [160]:
display_info( 'Steven', age=18 )
display_info( name='Sherry', age=15 )

'display_info' ran with arguments (Steven, 18)
Finished 'display_info' in 1.0006 secs
'display_info' ran with arguments (Sherry, 15)
Finished 'display_info' in 1.0007 secs


再加两个用户 Steven 和 Sherry，这时日志文件里的记录已经更新了。